# **Exploratory and Data Analysis**

In [28]:
import pandas as pd
file = r'C:\Users\danie\OneDrive\Escritorio\Data Science\Santander_Project\Salida.xlsx'
df = pd.read_excel(file)
print("El total de comentarios es:",len(df['Comentario']))

df

El total de comentarios es: 698


,Cargo,Comentario,Ubicacion,Rate
0,ejecutiva call center,Del Santander puedo decir que es una empresa q...,Región Metropolitana,5.0
1,Ejecutivo de vox,Trabajar en el call center es algo muy distint...,"Santiago de Chile, Región Metropolitana",2.0
2,Ejecutivo Py,Hay que andar subando el lomo para quedarse. A...,"San Fernando, O'Higgins",1.0
3,Ejecutivo Comercial,Si de verdad te gusta el área comercial para h...,"Santiago de Chile, Región Metropolitana",4.0
4,Analista de Gestión,"empresa muy preocupada de sus colaboradores, a...","Santiago de Chile, Región Metropolitana",5.0
...,...,...,...,...
693,Analyst at Products & Collaterals Department,Achievements: Recognized for leadership and ab...,NaN,5.0
694,Analista de Riesgo Automotriz,"- Lo que más me gusto en ambiente laboral, el ...","Santiago de Chile, RM",5.0
695,Ejecutivo de ventas,Aprendi acerca de otro banco de la competencia...,"Santiago de Chile, RM",3.0
696,Asistente Administrativo Especializado,"Grato ambiente de trabajo, alto profesionalism...","Santiago de Chile, RM",5.0


**La distribución del rate es la siguiente**

In [2]:
df['Rate'].value_counts()

5.0    353
4.0    199
3.0     67
2.0     19
1.0     10
Name: Rate, dtype: int64

**Quiero llevar todo a clasificación binaria, así que lo que voy a hacer será mirar un poco los comentarios con 3 estrellas, puesto
que estos decidirán si son postivios o negativos. Definiremos nuestra decision boundary sobre estos comentarios**

In [3]:
df[df['Rate']=='3'].to_string()

C:\Users\danie\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


'Empty DataFrame\nColumns: [Cargo, Comentario, Ubicacion, Rate]\nIndex: []'

In [29]:
h_rate = df[df['Rate'] >= 4 ]
l_rate = df[df['Rate'] <= 2 ]
h_rate

,Cargo,Comentario,Ubicacion,Rate
0,ejecutiva call center,Del Santander puedo decir que es una empresa q...,Región Metropolitana,5.0
3,Ejecutivo Comercial,Si de verdad te gusta el área comercial para h...,"Santiago de Chile, Región Metropolitana",4.0
4,Analista de Gestión,"empresa muy preocupada de sus colaboradores, a...","Santiago de Chile, Región Metropolitana",5.0
5,Ejecutivo de Cuentas,Fue una gran experiencia trabajar en Banco San...,"Santiago de Chile, Región Metropolitana",5.0
6,Cajera,La experiencia durante el trabajo me ayudó a c...,"Estación Central, Región Metropolitana",5.0
...,...,...,...,...
692,Ejecutivo Normalizaciòn,"personal altamente calificado , buena experien...","Santiago de Chile, RM",5.0
693,Analyst at Products & Collaterals Department,Achievements: Recognized for leadership and ab...,NaN,5.0
694,Analista de Riesgo Automotriz,"- Lo que más me gusto en ambiente laboral, el ...","Santiago de Chile, RM",5.0
696,Asistente Administrativo Especializado,"Grato ambiente de trabajo, alto profesionalism...","Santiago de Chile, RM",5.0




**Creamos un cuerpo de texto, en el idioma de NLP esto se llama Corpus**



In [30]:
corpus_high = [comentario for comentario in h_rate['Comentario']]
corpus_low = [comentario for comentario in l_rate['Comentario']]
corpus_high

['Del Santander puedo decir que es una empresa que vela mucho por su personal posee buenos beneficios y una plataforma excelente',
 'Si de verdad te gusta el área comercial para hacer carrera, es recomendable',
 'empresa muy preocupada de sus colaboradores, ambiente grato',
 'Fue una gran experiencia trabajar en Banco Santander Chile, me nutrieron de conocimientos en el área Bancaria. Siempre preocupados de mantenernos actualizados de los avances económicos y nuevas técnicas de ventas y asesoramiento al cliente.',
 'La experiencia durante el trabajo me ayudó a crecer profesional y laboralmente, ya que me encuentro estudiando y conocí mucho más en profundidad como es el sistema de inversiones y conocimiento de divisas.',
 'Una de las mejores empresas para trabajar en Chile, aunque aún existe burocracia interna y si tu trabajo es bueno pero no le caes bien a un superior, simplemente no te dan la oportunidad de crecer',
 'Aprendí muchísimo, totalmente agradecida de la oportunidad de conoc

# LDA model and data preprocessing

In [5]:
import nltk
from nltk import FreqDist
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
import pandas as pd

#pd.set_option("display.max_colwidth", 200)

import numpy as np
import re
import spacy
import gensim
from gensim import corpora

h_rate['Comentario'] = h_rate['Comentario'].str.replace("[^a-zA-Z#]", " ")
l_rate['Comentario'] = l_rate['Comentario'].str.replace("[^a-zA-Z#]", " ")

from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')




<ipython-input-31-cbfac13255d4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_rate['Comentario'] = h_rate['Comentario'].str.replace("[^a-zA-Z#]", " ")
<ipython-input-31-cbfac13255d4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_rate['Comentario'] = l_rate['Comentario'].str.replace("[^a-zA-Z#]", " ")


In [32]:
def word_freq(x, use = 30):
    
    
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()

    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
    return words_df

word_freq(h_rate['Comentario']) 





,word,count
0,Del,31
1,Santander,67
2,puedo,36
3,decir,35
4,que,349
...,...,...
2516,perdidas,1
2517,economicas,1
2518,sanciones,1
2519,organismos,1


In [8]:
def remove_stopwords(com):
    com_new = " ".join([i for i in com if i not in stop_words])
    return com_new


h_rate['Comentario'] = h_rate['Comentario'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# remove stopwords from the text
comentarios = [remove_stopwords(r.split()) for r in h_rate['Comentario']]

# make entire text lowercase
comentarios = [r.lower() for r in comentarios]



<ipython-input-8-3fdae4fd6714>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_rate['Comentario'] = h_rate['Comentario'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))


In [24]:
remove_stopwords(comentarios)
word_freq(comentarios)

,word,count
0,del,31
1,santander,81
2,puedo,36
3,decir,35
4,empresa,206
...,...,...
2118,perdidas,1
2119,economicas,1
2120,sanciones,1
2121,organismos,1


In [25]:
nlp = spacy.load('en_core_web_sm')

def lemmatization(texts, tags=['NOUN', 'ADJ']): # filter noun and adjective
       output = []
       for sent in texts:
             doc = nlp(" ".join(sent)) 
             output.append([token.lemma_ for token in doc if token.pos_ in tags])
       return output



C:\Users\danie\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [26]:
tokenized_comments = pd.Series(comentarios).apply(lambda x: x.split())
print(tokenized_comments[1])

['verdad', 'gusta', 'rea', 'comercial', 'hacer', 'carrera', 'recomendable']


In [33]:
comentarios_2 = lemmatization(tokenized_comments)
print(comentarios_2[1])

['hacer', 'carrera']


In [40]:
dictionary = corpora.Dictionary(tokenized_comments)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_comments]
LDA = gensim.models.ldamodel.LdaModel


lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,
                chunksize=1000, passes=50)

lda_model.print_topics()





[(0,
  '0.024*"trabajo" + 0.017*"ambiente" + 0.016*"laboral" + 0.014*"empresa" + 0.014*"mas" + 0.013*"buen" + 0.010*"banco" + 0.009*"cada" + 0.008*"equipo" + 0.007*"trabajadores"'),
 (1,
  '0.019*"empresa" + 0.018*"beneficios" + 0.014*"laboral" + 0.010*"sistema" + 0.009*"banco" + 0.008*"trabajo" + 0.008*"excelente" + 0.008*"buen" + 0.007*"clientes" + 0.007*"trabajar"'),
 (2,
  '0.030*"laboral" + 0.020*"empresa" + 0.020*"ambiente" + 0.018*"experiencia" + 0.017*"excelente" + 0.016*"buena" + 0.012*"buen" + 0.010*"trabajo" + 0.009*"santander" + 0.008*"gran"'),
 (3,
  '0.019*"trabajar" + 0.013*"banco" + 0.013*"empresa" + 0.012*"buena" + 0.010*"santander" + 0.009*"experiencia" + 0.007*"trabajo" + 0.007*"crecer" + 0.007*"conocimientos" + 0.007*"solo"'),
 (4,
  '0.041*"empresa" + 0.040*"excelente" + 0.032*"buenos" + 0.031*"beneficios" + 0.026*"santander" + 0.020*"posee" + 0.020*"personal" + 0.019*"decir" + 0.019*"puedo" + 0.018*"plataforma"'),
 (5,
  '0.025*"empresa" + 0.015*"gran" + 0.013*"tr